**PROJET DATAMINING**  -  *YANIS PIRES PORTELADA - SANJAY CANDA*

**ETAPE 1 :** Récupération de données à partir de **Wikidata**
Dans cette première étape, nous allons récupérer des données à partir de Wikidata en utilisant **SPARQL.**
Nous allons interroger **l'endpoint** de Wikidata pour obtenir une liste de *grandes villes, leurs pays et une image associée*.

In [36]:
import numpy as np
import pandas as pd
import matplotlib as plot
import SPARQLWrapper as sw
import os

In [37]:
import sys
from SPARQLWrapper import SPARQLWrapper, JSON

endpoint_url = "https://query.wikidata.org/sparql"

nombre_villes = 100

# Get cities
def get_cities(nombre_villes):
    query = """SELECT DISTINCT ?grandeville ?grandevilleLabel ?pays ?paysLabel ?image {
    ?grandeville wdt:P31 wd:Q1549591;
                wdt:P17 ?pays;
                wdt:P18 ?image.
    SERVICE wikibase:label { bd:serviceParam wikibase:language "fr". }
    }
    LIMIT """ + str(nombre_villes)
    return query

query = get_cities(nombre_villes)

def get_results(endpoint_url, query):
    user_agent = "WDQS-example Python/%s.%s" % (
        sys.version_info[0],
        sys.version_info[1],
    )
    sparql = SPARQLWrapper(endpoint_url, agent=user_agent)
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    return sparql.query().convert()


array = []
results = get_results(endpoint_url, query)

for result in results["results"]["bindings"]:
    array.append(
        (
            result["grandevilleLabel"]["value"],
            result["paysLabel"]["value"],
            result["image"]["value"],
        )
    )


"""-------------------------------------------------------------------------
Affiche le dataframe contenant les villes, les pays et les images associées.
----------------------------------------------------------------------------"""


dataframe = pd.DataFrame(array, columns=["ville", "pays", "image"])
dataframe = dataframe.astype(
    dtype={"ville": "<U200", "pays": "<U200", "image": "<U200"}
)


#affichage du dataframe
dataframe

,ville,pays,image
0,Krefeld,Allemagne,http://commons.wikimedia.org/wiki/Special:File...
1,Francfort-sur-le-Main,Allemagne,http://commons.wikimedia.org/wiki/Special:File...
2,Malmö,Suède,http://commons.wikimedia.org/wiki/Special:File...
3,Helsinki,Finlande,http://commons.wikimedia.org/wiki/Special:File...
4,Innsbruck,Autriche,http://commons.wikimedia.org/wiki/Special:File...
...,...,...,...
95,Cochin,Inde,http://commons.wikimedia.org/wiki/Special:File...
96,Potsdam,Allemagne,http://commons.wikimedia.org/wiki/Special:File...
97,Nassau,Bahamas,http://commons.wikimedia.org/wiki/Special:File...
98,Mönchengladbach,Allemagne,http://commons.wikimedia.org/wiki/Special:File...


**ETAPE 2 :** Téléchargement des données

In [38]:
import requests
import shutil
import time

def download_image(url):
    
    time.sleep(1) # Pause de 1 seconde entre les téléchargements pour éviter de surcharger le serveur de Wikidata
    
    headers = {"User-Agent": "WikidataImageDownloader/1.0 (contact: yanis.pires2004@gmail.com)"} # Permet d'éviter d'être bloqué par le serveur de Wikidata en raison d'un taux de requêtes élevé
    request = requests.get(url, allow_redirects=True, headers=headers, stream=True)
    
    if request.status_code == 200:
        os.makedirs("./images", exist_ok=True)
        filepath = os.path.join("./images", os.path.basename(url))
        
        with open(filepath, "wb") as image:
            request.raw.decode_content = True
            shutil.copyfileobj(request.raw, image)
            
      # Pause de 1 seconde entre les téléchargements
    return request.status_code


dataframe["filename"] = dataframe.image.apply(os.path.basename)
dataframe["filename"].value_counts()


# On vérifie si le dossier existe ET s'il contient des fichiers
if not os.path.exists("./images") or len(os.listdir("./images")) == 0:
    print("Dossier vide ou inexistant. Lancement du téléchargement...")
    dataframe.image.apply(download_image)
else:
    print(f"Le dossier contient déjà {len(os.listdir('./images'))} fichiers. Téléchargement annulé.")


dataframe

Dossier vide ou inexistant. Lancement du téléchargement...


,ville,pays,image,filename
0,Krefeld,Allemagne,http://commons.wikimedia.org/wiki/Special:File...,Rathaus%20Krefeld%20%288676439516%29%20v2.jpg
1,Francfort-sur-le-Main,Allemagne,http://commons.wikimedia.org/wiki/Special:File...,Frankfurter%20Altstadt%20mit%20Skyline%202019%...
2,Malmö,Suède,http://commons.wikimedia.org/wiki/Special:File...,Centrala%20Malm%C3%B6.jpg
3,Helsinki,Finlande,http://commons.wikimedia.org/wiki/Special:File...,Helsinki%20Lutheran%20Cathedral%20%28panoraama...
4,Innsbruck,Autriche,http://commons.wikimedia.org/wiki/Special:File...,Innsbruck%20-%20panoramio%20%2845%29.jpg
...,...,...,...,...
95,Cochin,Inde,http://commons.wikimedia.org/wiki/Special:File...,Kochi%20India.jpg
96,Potsdam,Allemagne,http://commons.wikimedia.org/wiki/Special:File...,2018-08-10%20DE%20Potsdam%2C%20Havel%2C%20Pots...
97,Nassau,Bahamas,http://commons.wikimedia.org/wiki/Special:File...,BHA%20Nassau%2C%20Paradise%20Island%2C%20Atlan...
98,Mönchengladbach,Allemagne,http://commons.wikimedia.org/wiki/Special:File...,M%C3%B6nchengladbach%20M%C3%BCnster%20St.%20Vi...


In [39]:
# récupère les métadonnées d'une image dans le dossier ./images
def get_image_metadata(filepath):
    from PIL import Image
    import os
    
    try:
        with Image.open(filepath) as img:
            width, height = img.size
            file_size = os.path.getsize(filepath)
            return {"width": width, "height": height, "file_size": file_size}
    except Exception as e:
        print(f"Error processing {filepath}: {e}")
        return None
        
# Applique la fonction get_image_metadata aux 100 premières images du dataframe
dataframe["metadata"] = dataframe.image.apply(lambda url: get_image_metadata(os.path.join("./images", os.path.basename(url))))
dataframe["metadata"].head(10)

#TODO séparer les métadonnées en colonnes distinctes (width, height, file_size).
#TODO recuperer la taille (petit, grand) et le format (paysage, portrait) des images en comparant les métadonnées.

c:\Users\Utilisateur 1\Desktop\Cours\Semestre 4\DataMining\.venv\Lib\site-packages\PIL\Image.py:3451: DecompressionBombWarning: Image size (99998272 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(


0    {'width': 4944, 'height': 2792, 'file_size': 4...
1    {'width': 13024, 'height': 7678, 'file_size': ...
2    {'width': 2932, 'height': 2932, 'file_size': 2...
3    {'width': 2373, 'height': 1557, 'file_size': 2...
4    {'width': 4667, 'height': 3111, 'file_size': 1...
5    {'width': 1024, 'height': 768, 'file_size': 14...
6    {'width': 5325, 'height': 3550, 'file_size': 3...
7    {'width': 4894, 'height': 3264, 'file_size': 8...
8    {'width': 1024, 'height': 683, 'file_size': 32...
9    {'width': 4423, 'height': 2969, 'file_size': 5...
Name: metadata, dtype: object

In [40]:
%pip install webcolors

Note: you may need to restart the kernel to use updated packages.


In [41]:
from PIL import Image
import numpy as np
from sklearn.cluster import KMeans
import webcolors
import os

# Trouver la couleur CSS3 la plus proche
def closest_color(requested_color):
    min_distance = float("inf")
    closest_name = None

    for name in webcolors.names("css3"):
        r_c, g_c, b_c = webcolors.name_to_rgb(name)
        distance = (
            (r_c - requested_color[0]) ** 2 +
            (g_c - requested_color[1]) ** 2 +
            (b_c - requested_color[2]) ** 2
        )

        if distance < min_distance:
            min_distance = distance
            closest_name = name

    return closest_name


# Convertir RGB en nom
def get_color_name(rgb_color):
    try:
        return webcolors.rgb_to_name(rgb_color, spec="css3")
    except ValueError:
        return closest_color(rgb_color)


def get_dominant_colors(filepath, num_colors=1):
    try:
        with Image.open(filepath) as img:
            img = img.convert("RGB")
            img = img.resize((150, 150))

            img_array = np.array(img).reshape(-1, 3)

            kmeans = KMeans(n_clusters=num_colors, n_init=10, random_state=42)
            kmeans.fit(img_array)

            colors = kmeans.cluster_centers_.astype(int)

            color_names = [
                get_color_name(tuple(color))
                for color in colors
            ]

            return color_names

    except Exception as e:
        print(f"Error processing {filepath}: {e}")
        return None


# Application à la dataframe
dataframe["dominant_colors"] = dataframe.image.apply(
    lambda url: get_dominant_colors(
        os.path.join("./images", os.path.basename(url))
    )
)

dataframe.head()


c:\Users\Utilisateur 1\Desktop\Cours\Semestre 4\DataMining\.venv\Lib\site-packages\PIL\Image.py:3451: DecompressionBombWarning: Image size (99998272 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(


,ville,pays,image,filename,metadata,dominant_colors
0,Krefeld,Allemagne,http://commons.wikimedia.org/wiki/Special:File...,Rathaus%20Krefeld%20%288676439516%29%20v2.jpg,"{'width': 4944, 'height': 2792, 'file_size': 4...",[darkgray]
1,Francfort-sur-le-Main,Allemagne,http://commons.wikimedia.org/wiki/Special:File...,Frankfurter%20Altstadt%20mit%20Skyline%202019%...,"{'width': 13024, 'height': 7678, 'file_size': ...",[lightslategray]
2,Malmö,Suède,http://commons.wikimedia.org/wiki/Special:File...,Centrala%20Malm%C3%B6.jpg,"{'width': 2932, 'height': 2932, 'file_size': 2...",[darkslategray]
3,Helsinki,Finlande,http://commons.wikimedia.org/wiki/Special:File...,Helsinki%20Lutheran%20Cathedral%20%28panoraama...,"{'width': 2373, 'height': 1557, 'file_size': 2...",[dimgray]
4,Innsbruck,Autriche,http://commons.wikimedia.org/wiki/Special:File...,Innsbruck%20-%20panoramio%20%2845%29.jpg,"{'width': 4667, 'height': 3111, 'file_size': 1...",[gray]


In [43]:
dataframe[dataframe["filename"] == "Concepcion%2C%20vista%20de%20Chepen%20%2813654156695%29.jpg"]

,ville,pays,image,filename,metadata,dominant_colors
60,Concepción,Chili,http://commons.wikimedia.org/wiki/Special:File...,Concepcion%2C%20vista%20de%20Chepen%20%2813654...,"{'width': 4000, 'height': 3000, 'file_size': 1...",[slategray]
